In [814]:
import cirq
import numpy as np
import random
import sympy
from math import pi
import time
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from pennylane.optimize import NesterovMomentumOptimizer

%matplotlib inline

np.random.seed(42)

In [809]:
def U_phi(q, W):
    for i in range(len(q)):
        rot = cirq.ZPowGate(exponent=W[i]/pi)
        yield rot(q[i])
    for i in range(len(q)-1):
        for j in range(i+1,len(q)):
            rot = cirq.ZPowGate(exponent=((pi-W[i])*(pi-W[j]))/pi)
            yield rot.on(q[j]).controlled_by(q[i])
            
def fancy_U(q, W):
    for i in range(len(q)):
        yield cirq.H(q[i])
    yield U_phi(q, W)
    for i in range(len(q)):
        yield cirq.H(q[i])
    yield U_phi(q, W)

def W_theta(q, theta):
    for i in range(len(q)):
        yield cirq.CZ.on(q[(i+1)%len(q)],q[i])
    for i in range(len(q)):
        rot = cirq.ZPowGate(exponent=theta[2*i]/pi)
        yield rot(q[i])
        rot = cirq.Ry(theta[2*i+1])
        yield rot(q[i])

def measure(q):
    for i in range(len(q)):
        yield cirq.measure(q[i], key=str(i))

In [810]:
def circuit(q, W, theta, layers):
    yield fancy_U(q,W)
    for i in range(len(q)):
        rot = cirq.ZPowGate(exponent = theta[2*i]/pi)
        yield rot(q[i])
        rot = cirq.Ry(theta[2*i + 1])
        yield rot(q[i])
    
    for i in range(1,layers+1):
        yield W_theta(q, theta[range(6*(i),6*(i+1))])
    yield measure(q)

In [811]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p)**2
    loss = loss/ len(labels)
    return loss

def accuracy(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)
    return loss

def prediction(results):
    counter = (results.multi_measurement_histogram(keys="012"))
    p_hold = 0
    for j in counter:
        if j.count(1) % 2 == 1:
            p_hold += counter[j] 
    return p_hold/shots

def variational_classifier(q, x, theta, l, shots):
    simulator = cirq.Simulator()
    c = cirq.Circuit()
    c.append(circuit(q, x, theta, l))
    return simulator.run(c, repetitions=shots)
        
def cost(theta, X, Y, q, l, batch_i, shots, key):
    p = []
    for i in batch_i:
        results = variational_classifier(q, X[i], theta, l, shots)
        p.append(prediction(results))
    return square_loss(Y[batch_i], p)

In [812]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
	"""
	Call in a loop to create terminal progress bar
	@params:
		iteration   - Required  : current iteration (Int)
		total	   - Required  : total iterations (Int)
		prefix	  - Optional  : prefix string (Str)
		suffix	  - Optional  : suffix string (Str)
		decimals	- Optional  : positive number of decimals in percent complete (Int)
		length	  - Optional  : character length of bar (Int)
		fill		- Optional  : bar fill character (Str)
	"""
	percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
	filledLength = int(length * iteration // total)
	bar = fill * filledLength + '-' * (length - filledLength)
	print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
	# Print New Line on Complete
	if iteration == total: 
		print()

In [829]:
nr_qubits = 3
nr_layers = 1
batch_size = 10
shots = 100
iterations = 200
key = ""
for i in range(nr_qubits):
    key += str(i)

#Set up input and output qubits.
qubits = [cirq.GridQubit(i, 0) for i in range(nr_qubits)]

df = pd.read_csv("QA_data.csv")
X = df.iloc[:,:3].to_numpy()
Y = df.iloc[:,3].to_numpy()

m = int(np.round((2/3)*len(X)))
train = np.random.randint(0, len(X), (m,))
X_t = X[train,:]
Y_t = Y[train]

nr_par = (nr_qubits*2)*(nr_layers+1)
init_theta = np.array(0.01*np.random.rand(nr_par,)*(2*pi))
theta = init_theta
iz = np.array(range(m))
eye = np.eye(nr_par)
c = 0.1
a = 2*pi*0.1
alpha = 0.602
gamma = 0.101
plot_ix = 10
P = int(iterations/plot_ix)
tot_loss = np.zeros(iterations)
opt = NesterovMomentumOptimizer(0.5)
for k in range(1,iterations+1):
    printProgressBar((k-1)*nr_par, iterations*nr_par, prefix = 'Progress:', suffix = "Complete | acc="+str(round(acc,3)), length = 50)
    batch_ix = np.random.randint(0, len(X), (batch_size,))
    c_n = c/(k**(0.602))
    a_n = a/(k**(0.101))
    gradient = np.zeros(nr_par)
    for i in range(nr_par):
        printProgressBar((k-1)*nr_par+i, iterations*nr_par, prefix = 'Progress:', suffix = 'Complete', length = 50)
        start = time.time()
        loss_plus = cost(theta+c_n*eye[:,i], X, Y, qubits, nr_layers, batch_ix, shots, key)
        loss_minus= cost(theta-c_n*eye[:,i], X, Y, qubits, nr_layers, batch_ix, shots, key)
        gradient[i] = (loss_plus - loss_minus)/(2*c_n)
        end = time.time()
#     theta = opt.step(lambda v: cost(v, X_t, Y_t, qubits, nr_layers, iz, shots, key), theta)
    theta = (theta - a_n*gradient) % (2*pi)
    tot_loss[k-1] = cost(theta, X_t, Y_t, qubits, nr_layers, iz, shots, key)
    predictions = [np.sign(prediction(variational_classifier(qubits, x, theta, nr_layers, int(shots/10)))) for x in X]
    acc = accuracy(Y, predictions)
    
printProgressBar(iterations*nr_par, iterations*nr_par, prefix = 'Progress:', suffix = 'Complete', length = 50)

print("init_theta - end_theta: ")
print(np.around(init_theta-theta,3))
print("gradient: ")
print(gradient)
fig = plt.figure(figsize=(15,10))
ax = plt.gca()
plt.plot(range(1,iterations+1), tot_loss, 'g-', markersize=2)

KeyboardInterrupt: 